In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [4]:
columns = ['age','sex','cp','trestbps','chol','fbs','restecg',
           'thalach','exang','oldpeak','slope','ca','thal','num']

data = np.zeros((1, len(columns)))

for file in ['../UCI_data/processed.va.csv', '../UCI_data/processed.switzerland.csv', 
             '../UCI_data/processed.cleveland.csv', '../UCI_data/processed.hungarian.csv']:
    data = np.vstack((data, np.genfromtxt(file, delimiter=',')))
data = data[1:]
df = pd.DataFrame(data=data, columns=columns, index=None)

In [5]:
non_na_df = df[(~df.isna()).all(axis=1)]

In [34]:
X = non_na_df.drop(columns="num")
y = non_na_df["num"].astype(int)
y.loc[y > 0] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [135]:
class Encoder(layers.Layer):
    def __init__(self,
                 intermidiate_1_dim,
                 intermidiate_2_dim,
                 latent_dim,
                 name='encoder',
                 **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.batch_normalization_1 = layers.BatchNormalization()
        self.batch_normalization_2 = layers.BatchNormalization()
        
        self.inp = tf.keras.Input(shape=n)
        
        self.layer_1 = layers.Dense(intermidiate_1_dim, activation='tanh')
        self.layer_2 = layers.Dense(intermidiate_2_dim, activation='tanh')
        self.layer_3 = layers.Dense(latent_dim, activation='tanh')

    def call(self, inputs):
        bn_1 = self.batch_normalization_1(inputs)
        
        layer_1 = self.layer_1(bn_1)
        layer_2 = self.layer_2(layer_1)
        
        bn_2 = self.batch_normalization_2(layer_2)
        
        return self.layer_3(bn_2)


class Decoder(layers.Layer):
    def __init__(self,
                 intermidiate_1_dim,
                 intermidiate_2_dim,
                 original_dim,
                 name='decoder',
                 **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.batch_normalization_3 = layers.BatchNormalization()
        self.batch_normalization_4 = layers.BatchNormalization()
        
        self.layer_4 = layers.Dense(intermidiate_2_dim, activation='tanh')
        self.layer_5 = layers.Dense(intermidiate_1_dim, activation='tanh')
        self.layer_6 = layers.Dense(original_dim, activation='tanh')

    def call(self, inputs):
        bn_3 = self.batch_normalization_3(inputs)
        
        layer_4 = self.layer_4(bn_3)
        layer_5 = self.layer_5(layer_4)
        
        bn_4 = self.batch_normalization_4(layer_5)
        
        return self.layer_6(bn_4)


class BottleneckAutoEncoder(tf.keras.Model):
    def __init__(self,
                 original_dim,
                 name='autoencoder',
                 **kwargs):
        super(BottleneckAutoEncoder, self).__init__(name=name, **kwargs)
        
        self.original_dim = original_dim
        
        intermidiate_1_dim = original_dim / 2
        intermidiate_2_dim = original_dim / 4
        latent_dim = 2
        
        self.encoder = Encoder(
            intermidiate_1_dim=intermidiate_1_dim,
            intermidiate_2_dim=intermidiate_2_dim,
            latent_dim=latent_dim
        )
        self.decoder = Decoder(
            intermidiate_1_dim=intermidiate_1_dim,
            intermidiate_2_dim=intermidiate_2_dim,
            original_dim=original_dim
        )

    def call(self, inputs):
        latent = self.encoder(inputs)
        
        self.latent = latent
        
        reconstructed = self.decoder(latent)
        return reconstructed
    
    def get_latent_layer(self):
        return self.latent

In [147]:
bae = BottleneckAutoEncoder(len(X.columns))

In [148]:
Adam_Opt = tf.keras.optimizers.Adam(lr=l2_reg)
bae.compile(optimizer=Adam_Opt,
            loss="mean_squared_error",
            metrics=["mean_squared_error"])

In [149]:
bae.fit(
    tf.constant(X_train.values, dtype = tf.float32, shape=X_train.shape), 
    tf.constant(X_train.values, dtype = tf.float32, shape=X_train.shape), 
    epochs=200,
    validation_data=(
        tf.constant(X_test.values, dtype = tf.float32, shape=X_test.shape),
        tf.constant(X_test.values, dtype = tf.float32, shape=X_test.shape)
    )
)

Train on 239 samples, validate on 60 samples
Epoch 1/200
239/239 [==============================] - 2s 7ms/sample - loss: 8295.8730 - mean_squared_error: 8295.8730 - val_loss: 8083.0335 - val_mean_squared_error: 8083.0332
Epoch 2/200
239/239 [==============================] - 0s 229us/sample - loss: 8295.6407 - mean_squared_error: 8295.6406 - val_loss: 8083.0342 - val_mean_squared_error: 8083.0342
Epoch 3/200
239/239 [==============================] - 0s 230us/sample - loss: 8295.5219 - mean_squared_error: 8295.5225 - val_loss: 8083.1028 - val_mean_squared_error: 8083.1025
Epoch 4/200
239/239 [==============================] - 0s 208us/sample - loss: 8295.1649 - mean_squared_error: 8295.1660 - val_loss: 8083.1900 - val_mean_squared_error: 8083.1895
Epoch 5/200
239/239 [==============================] - 0s 211us/sample - loss: 8295.2086 - mean_squared_error: 8295.2090 - val_loss: 8083.2624 - val_mean_squared_error: 8083.2627
Epoch 6/200
239/239 [==============================] - 0s 244u

239/239 [==============================] - 0s 246us/sample - loss: 8287.4897 - mean_squared_error: 8287.4893 - val_loss: 8069.6623 - val_mean_squared_error: 8069.6626
Epoch 47/200
239/239 [==============================] - 0s 234us/sample - loss: 8287.2628 - mean_squared_error: 8287.2627 - val_loss: 8069.3479 - val_mean_squared_error: 8069.3481
Epoch 48/200
239/239 [==============================] - 0s 214us/sample - loss: 8286.9818 - mean_squared_error: 8286.9814 - val_loss: 8069.1202 - val_mean_squared_error: 8069.1196
Epoch 49/200
239/239 [==============================] - 0s 203us/sample - loss: 8286.8843 - mean_squared_error: 8286.8838 - val_loss: 8068.8535 - val_mean_squared_error: 8068.8530
Epoch 50/200
239/239 [==============================] - 0s 208us/sample - loss: 8286.7739 - mean_squared_error: 8286.7744 - val_loss: 8068.6237 - val_mean_squared_error: 8068.6240
Epoch 51/200
239/239 [==============================] - 0s 201us/sample - loss: 8286.3511 - mean_squared_error: 8

239/239 [==============================] - 0s 171us/sample - loss: 8277.9440 - mean_squared_error: 8277.9443 - val_loss: 8061.5357 - val_mean_squared_error: 8061.5356
Epoch 92/200
239/239 [==============================] - 0s 190us/sample - loss: 8277.7618 - mean_squared_error: 8277.7617 - val_loss: 8061.3632 - val_mean_squared_error: 8061.3628
Epoch 93/200
239/239 [==============================] - 0s 191us/sample - loss: 8277.5573 - mean_squared_error: 8277.5576 - val_loss: 8061.1818 - val_mean_squared_error: 8061.1816
Epoch 94/200
239/239 [==============================] - 0s 209us/sample - loss: 8277.3012 - mean_squared_error: 8277.3008 - val_loss: 8061.0140 - val_mean_squared_error: 8061.0137
Epoch 95/200
239/239 [==============================] - 0s 200us/sample - loss: 8277.1218 - mean_squared_error: 8277.1211 - val_loss: 8060.8146 - val_mean_squared_error: 8060.8145
Epoch 96/200
239/239 [==============================] - 0s 201us/sample - loss: 8276.8486 - mean_squared_error: 8

239/239 [==============================] - 0s 167us/sample - loss: 8267.8856 - mean_squared_error: 8267.8857 - val_loss: 8052.6116 - val_mean_squared_error: 8052.6113
Epoch 137/200
239/239 [==============================] - 0s 200us/sample - loss: 8267.6444 - mean_squared_error: 8267.6436 - val_loss: 8052.3635 - val_mean_squared_error: 8052.3638
Epoch 138/200
239/239 [==============================] - 0s 202us/sample - loss: 8267.4753 - mean_squared_error: 8267.4746 - val_loss: 8052.1006 - val_mean_squared_error: 8052.1006
Epoch 139/200
239/239 [==============================] - 0s 178us/sample - loss: 8267.2665 - mean_squared_error: 8267.2666 - val_loss: 8051.8434 - val_mean_squared_error: 8051.8433
Epoch 140/200
239/239 [==============================] - 0s 166us/sample - loss: 8267.0427 - mean_squared_error: 8267.0420 - val_loss: 8051.6309 - val_mean_squared_error: 8051.6309
Epoch 141/200
239/239 [==============================] - 0s 198us/sample - loss: 8266.7885 - mean_squared_err

239/239 [==============================] - 0s 167us/sample - loss: 8257.0995 - mean_squared_error: 8257.0996 - val_loss: 8041.5182 - val_mean_squared_error: 8041.5181
Epoch 182/200
239/239 [==============================] - 0s 184us/sample - loss: 8256.9218 - mean_squared_error: 8256.9219 - val_loss: 8041.2775 - val_mean_squared_error: 8041.2769
Epoch 183/200
239/239 [==============================] - 0s 198us/sample - loss: 8256.6615 - mean_squared_error: 8256.6611 - val_loss: 8041.0222 - val_mean_squared_error: 8041.0225
Epoch 184/200
239/239 [==============================] - 0s 171us/sample - loss: 8256.4329 - mean_squared_error: 8256.4326 - val_loss: 8040.8444 - val_mean_squared_error: 8040.8447
Epoch 185/200
239/239 [==============================] - 0s 209us/sample - loss: 8256.1775 - mean_squared_error: 8256.1777 - val_loss: 8040.5970 - val_mean_squared_error: 8040.5967
Epoch 186/200
239/239 [==============================] - 0s 183us/sample - loss: 8255.9280 - mean_squared_err

In [93]:
y_pred = np.argmax(model.predict(tf.constant(X_test.values, dtype = tf.float32, shape=X_test.shape)), axis=1)

In [95]:
roc_auc_score(y_test.values, y_pred), accuracy_score(y_test.values, y_pred), f1_score(y_test.values, y_pred) 

(0.5707070707070707, 0.5833333333333334, 0.6478873239436619)

In [152]:
bae(tf.constant(X.values, dtype = tf.float32, shape=X.shape))

<tf.Tensor: id=117903, shape=(299, 13), dtype=float32, numpy=
array([[ 0.28835478,  0.14486174,  0.4832785 , ..., -0.11703872,
         0.05467898,  0.518913  ],
       [ 0.03485138,  0.43346864, -0.4023548 , ...,  0.6122683 ,
         0.62816334, -0.49825034],
       [-0.06248675,  0.5431028 , -0.16400601, ...,  0.41417527,
         0.50028086, -0.45568088],
       ...,
       [ 0.29214686,  0.13922442,  0.47407654, ..., -0.10471132,
         0.0624101 ,  0.5170502 ],
       [ 0.36040697,  0.03977748,  0.23065975, ...,  0.17707574,
         0.24233076,  0.45218962],
       [-0.07093074,  0.55709326, -0.07928354, ...,  0.343726  ,
         0.43524945, -0.40502462]], dtype=float32)>

In [153]:
bae.get_latent_layer()

<tf.Tensor: id=117858, shape=(299, 2), dtype=float32, numpy=
array([[ 2.36965805e-01, -9.91128027e-01],
       [-5.11072241e-02,  9.74943161e-01],
       [ 2.18536016e-02,  9.46736932e-01],
       [ 6.07715361e-02,  2.00155467e-01],
       [ 6.68314248e-02,  7.42450297e-01],
       [ 1.56383216e-01, -6.49222910e-01],
       [ 2.04190254e-01, -9.81466591e-01],
       [ 4.38859537e-02,  9.15878952e-01],
       [ 9.38637257e-02, -1.15944847e-01],
       [ 5.18920608e-02,  2.34975427e-01],
       [-1.11010056e-02,  9.43449318e-01],
       [ 2.00454339e-01, -9.46290135e-01],
       [-2.53561810e-02,  9.57427323e-01],
       [ 1.00601120e-02,  3.23259503e-01],
       [ 1.65159792e-01, -9.53274488e-01],
       [ 1.96775660e-01, -9.05523419e-01],
       [ 2.24821687e-01, -9.44415033e-01],
       [ 1.94909677e-01, -9.67077017e-01],
       [ 1.76158771e-01, -8.88261199e-01],
       [ 9.24243405e-02, -1.35587290e-01],
       [ 1.73688769e-01, -9.38286185e-01],
       [ 1.85741857e-01, -8.78505707